In [198]:
import pandas as pd
import numpy as np
import sklearn.cluster
import distance
import csv
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from collections import Counter
import pprint
from nltk.stem import WordNetLemmatizer
from dateutil import parser as dateParser
import time

In [199]:
pp = pprint.PrettyPrinter(indent=4)
wnl = WordNetLemmatizer()

In [200]:
with open('AnalysisData.csv') as data_file:    
    data = csv.DictReader(data_file, delimiter=',', quotechar='"')
    data.next
raw_data = open('AnalysisData.csv', 'rb')
reader = csv.reader(raw_data, delimiter=',', quoting=csv.QUOTE_MINIMAL)
x = list(reader)
for i in xrange(len(x)):
    del x[i][4]
    del x[i][3]
    x[i][0] = int(time.mktime(dateParser.parse(x[i][0]).timetuple()))

xx = x[:]

http://chrisalbon.com/python/cleaning_text.html

In [201]:
for i in xrange(len(x)):
    from_word = x[i][1]
    
    if '-' in from_word and from_word.index('-') == 2:
        x[i][1] = from_word[3:].lower()

In [202]:
regex = re.compile('[%s]' % re.escape(string.punctuation))
for i in xrange(len(x)):
    
    tokenized_sms = word_tokenize(x[i][2])
    tokenized_sms_optimized = []
    
    for tokenized_word in tokenized_sms:

        new_tokenized_word = []
        for tokenized_character in tokenized_word: 
            new_tokenized_character = regex.sub('', tokenized_character)
            if not new_tokenized_character == '':
                new_tokenized_word.append(new_tokenized_character)
        if new_tokenized_word:
            new_tokenized_word = "".join(new_tokenized_word).lower()
            if not new_tokenized_word in stopwords.words('english'):
                tokenized_sms_optimized.append(wnl.lemmatize(wnl.lemmatize(new_tokenized_word, 'v')))
    
    x[i][2] = " ".join(tokenized_sms_optimized)

In [203]:
df = pd.DataFrame(x, columns=['time', 'from', 'body'])

http://chrisalbon.com/python/pandas_list_unique_values_in_column.html

In [204]:
pp.pprint(df.shape)
from_unique_list = list(df.get('from').unique())
print("Unique Timestamps: " + str(len(df.get('time').unique())))
print("Unique From: " + str(len(from_unique_list)))
print("Unique SMS: " + str(len(df.get('body').unique())))

(6784, 3)
Unique Timestamps: 6768
Unique From: 835
Unique SMS: 5598


In [205]:
from_unique_dict = {i:x for i, x in enumerate(from_unique_list)}
from_unique_dict.update({x:i for i, x in enumerate(from_unique_list)})

if len(from_unique_dict) == len(from_unique_list)*2:
    print("Successfully created unique dictionary from FROM data of sms")
else:
    print("Some FROM data is equal to number assigned to it")

from_count = Counter(df.get('from'))
print("\nCount of sms from each sender")
#pp.pprint(from_count.most_common(n=30))

sms_words_count = Counter()
df.get('body').str.lower().str.split().apply(sms_words_count.update)
print("\nCount of words in all sms")
#pp.pprint(sms_words_count.most_common(n=30))

Successfully created unique dictionary from FROM data of sms

Count of sms from each sender

Count of words in all sms


In [206]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).quick_ratio()

In [207]:
df['category'] = 0
df['similar_to'] = -1
df['similarity_score'] = -1.0
category_list = ['other', 'bank', 'spam', 'ecomm', 'travel', 'personal', 'otp', 'notif', 'offer', 'alert']

In [208]:
#df.head()

In [209]:
def set_similar_sms_to_category(sms_id, category_name):
    
    category_id = category_list.index(category_name)
    main_sms = df['body'][sms_id]
    similar_sms_ids = [sms_id]
    
    for i in xrange(len(df.get('body'))):
        check_sms = df['body'][i]
        new_similarity_score = similar(main_sms, check_sms)
        
        if new_similarity_score > 0.8:
            if df['similar_to'][i] == -1:
                df.set_value(i, 'similar_to', sms_id)
                df.set_value(i, 'category', category_id)
                df.set_value(i, 'similarity_score', new_similarity_score)
            else:
                if new_similarity_score > df['similarity_score'][i]:
                    df.set_value(i, 'similar_to', sms_id)
                    df.set_value(i, 'category', category_id)
                    df.set_value(i, 'similarity_score', new_similarity_score)


In [210]:
def get_4_random_sms():
    return df.ix[np.random.choice(df.index, 4)]

category_list = ['other', 'bank', 'spam', 'ecomm', 'travel', 'personal', 'otp', 'notif', 'offer', 'alert']

In [211]:
random_sms = get_4_random_sms()

for x in random_sms.index:
    #print(xx[x][2])
    #category_name = raw_input()
    #set_similar_sms_to_category(x, category_name)

IndentationError: expected an indented block (<ipython-input-211-1f92157af34e>, line 6)

In [212]:
trained_list = []
for x in df.index:
    if df['similar_to'][x] != -1:
        li = list(df.ix[x])
        trained_list.append([li[1], category_list[li[3]], li[5], li[2]])

dff = pd.DataFrame(trained_list)